In [1]:
import matplotlib.pyplot as plt
import pycpd
import numpy as np
import pandas as pd
import os
import glob
import shutil
import open3d as o3d
import time
import ast
import h5py
from utils import rotationMatrixToEulerAngles

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
nameIndexMapCSV = r"./data/nameIndexMapping.csv"
dfNameIndexMap = pd.read_csv(nameIndexMapCSV)
rowCount = dfNameIndexMap.shape[0]
dfNameIndexMap.head(5)

,tag,srcObj,srcTxt
0,0L,./data/allData86Sample/caodongjuan_l.obj,./data/allData86Sample/caodongjuan_l.txt
1,0U,./data/allData86Sample/caodongjuan_u.obj,./data/allData86Sample/caodongjuan_u.txt
2,10L,./data/allData86Sample/caoxu_l.obj,./data/allData86Sample/caoxu_l.txt
3,10U,./data/allData86Sample/caoxu_u.obj,./data/allData86Sample/caoxu_u.txt
4,11L,./data/allData86Sample/axin_l.obj,./data/allData86Sample/axin_l.txt


In [3]:
nameIndexMapCSVSupp = r"./data-refined/nameIndexMapping.csv"
suppDataDir = r".\data-refined\repaired-txt"
suppStartIndex = 87 # indexing from 87U and 87L
dfNameIndexMapSupp = pd.read_csv(nameIndexMapCSVSupp)
rowCountSupp = dfNameIndexMapSupp.shape[0]
dfNameIndexMapSupp["tag"] = dfNameIndexMapSupp["tag"].apply(lambda x:str(int(x[:-1])+suppStartIndex)+x[-1])
dfNameIndexMapSupp.head(5)

,tag,cutMesh,cutResult,toothMesh,auxiliaryInfo
0,87L,143123_cutMesh_L.obj,143123_cutResult_L.txt,143123_toothMesh_L.obj,143123_alignLoc_Dir_L.txt
1,87U,143123_cutMesh_U.obj,143123_cutResult_U.txt,143123_toothMesh_U.obj,143123_alignLoc_Dir_U.txt
2,88L,162065_cutMesh_L.obj,162065_cutResult_L.txt,162065_toothMesh_L.obj,162065_alignLoc_Dir_L.txt
3,88U,162065_cutMesh_U.obj,162065_cutResult_U.txt,162065_toothMesh_U.obj,162065_alignLoc_Dir_U.txt
4,89L,166779_cutMesh_L.obj,166779_cutResult_L.txt,166779_toothMesh_L.obj,166779_alignLoc_Dir_L.txt


In [4]:
# 将前后两次得到的数据合并
dfNameIndexMapMerged = pd.merge(dfNameIndexMap, dfNameIndexMapSupp, how="outer", on="tag")
rowCountTotal = dfNameIndexMapMerged.shape[0]
assert rowCount + rowCountSupp == rowCountTotal
dfNameIndexMapMerged.head(5)

,tag,srcObj,srcTxt,cutMesh,cutResult,toothMesh,auxiliaryInfo
0,0L,./data/allData86Sample/caodongjuan_l.obj,./data/allData86Sample/caodongjuan_l.txt,NaN,NaN,NaN,NaN
1,0U,./data/allData86Sample/caodongjuan_u.obj,./data/allData86Sample/caodongjuan_u.txt,NaN,NaN,NaN,NaN
2,10L,./data/allData86Sample/caoxu_l.obj,./data/allData86Sample/caoxu_l.txt,NaN,NaN,NaN,NaN
3,10U,./data/allData86Sample/caoxu_u.obj,./data/allData86Sample/caoxu_u.txt,NaN,NaN,NaN,NaN
4,11L,./data/allData86Sample/axin_l.obj,./data/allData86Sample/axin_l.txt,NaN,NaN,NaN,NaN


In [5]:
# 分成上牙列和下压列
__dfLowerToothRow = dfNameIndexMapMerged[dfNameIndexMapMerged["tag"].apply(lambda x:x.upper()[-1]=="L")].copy()
__dfUpperToothRow = dfNameIndexMapMerged[dfNameIndexMapMerged["tag"].apply(lambda x:x.upper()[-1]=="U")].copy()
print("Tooth row sample num: {} upper, {} lower.".format(__dfUpperToothRow.shape[0],__dfLowerToothRow.shape[0]))

Tooth row sample num: 138 upper, 138 lower.


In [6]:
RootDir = r".\data\repaired-txt"
upperToothIndices = [11,12,13,14,15,16,17,21,22,23,24,25,26,27] #不考虑智齿18,28
lowerToothIndices = [31,32,33,34,35,36,37,41,42,43,44,45,46,47] #不考虑智齿38,48

In [7]:
def __getCentroidXYZ(row):
    if not os.path.exists(row["tPath"]):
        return np.nan, np.nan, np.nan
    else:
        centroid = np.loadtxt(row["tPath"]).mean(axis=0)
        return centroid[0], centroid[1], centroid[2]

CENTROID_START_INDEX = 7
def createToothRowCentroidsDF(srcDir, UorL, toothIndices, dfNameIndexMap, centroidStartIndex=CENTROID_START_INDEX):
    """计算每颗牙齿的初始重心坐标，记录在csv中"""
    dfToothRow = dfNameIndexMap[dfNameIndexMap["tag"].apply(lambda x:x.upper()[-1]==UorL)].copy()
    for uTId in toothIndices:
        dfToothRow["tPath"] = dfToothRow["tag"].apply(lambda tag:os.path.join(srcDir,str(uTId),tag+".txt"))
        newCols = [str(uTId)+"x",str(uTId)+"y",str(uTId)+"z"]
        dfToothRow[newCols] = dfToothRow.apply(__getCentroidXYZ, axis=1, result_type="expand")
    dfToothRow.drop(columns="tPath", inplace=True) #删除临时变量
    dfToothRow = dfToothRow[~dfToothRow.iloc[:,centroidStartIndex:].isnull().all(axis=1)] #删除全空的行
    dfToothRow.reset_index(drop=True, inplace=True)
    return dfToothRow

In [8]:
badInitDataIndices = {21:["65U",],31:["43L","85L"],36:["83L","84L"]}
# 需要跳过的补充数据（有缺陷的数据）
badSuppDataIndices = {11:[17,25,31], 12:[20,39,7], 13:[10,12,15,28,33], 14:[15,36,39], 15:[33,35,36],\
                     16:[12,13,20,24,29,30,32,35,36,40,41,43,47,9], 17:[11,13,14,17,21,22,24,26,27,2,32,40,41,43,45,47,50,51,9],\
                     21:[7,], 22:[], 23:[5,25], 24:[20,25,33,48], 25:[20,25,48], 26:[19,33,43,48], 27:[19,20,24,25,3,8],\
                     31:[11,20,39,51], 32:[20,33,48,51], 33:[11,12,20,22,33,48,51], 34:[16,], 35:[32,], 36:[24,], 37:[11,20,50],\
                     41:[11,20,48,51], 42:[20,36], 43:[11,12,20,25,2,39], 44:[25,2,30,36], 45:[30,35], 46:[15,2,35,43], 47:[15,35,43]}

In [9]:
def getFileTagWithoutUL(f):
    return int(os.path.basename(f).split(".")[0][:-1])

def getFileUL(f):
    return os.path.basename(f).split(".")[0][-1]

# # Run it only once
# # Merge suupplementary data into initial data
# for tID,badIndices in badSuppDataIndices.items():
#     allFiles = glob.glob(os.path.join(suppDataDir, str(tID), "*.txt"))
#     normalFiles = [f for f in allFiles if getFileTagWithoutUL(f) not in badIndices]
#     assert (len(allFiles) - len(normalFiles)) == len(badIndices)
#     dstFiles = [os.path.join(RootDir,str(tID),str(getFileTagWithoutUL(f)+suppStartIndex)+getFileUL(f)+".txt") for f in normalFiles]
#     for srcF,dstF in zip(normalFiles,dstFiles):
#         assert getFileTagWithoutUL(dstF) - getFileTagWithoutUL(srcF) == suppStartIndex
#         try:
#             shutil.copy(srcF, dstF)
#             # print("File copied successfully.")
#         # If source and destination are same
#         except shutil.SameFileError:
#             print("Source and destination represents the same file.")
#         # If there is any permission issue
#         except PermissionError:
#             print("Permission denied.")
#         # For other errors
#         except:
#             print("Error occurred while copying file.")

In [10]:
upperToothRowCentroidsCSV = r"./data/upperToothRowCentroids.csv"
# dfUpperToothRow = createToothRowCentroidsDF(RootDir, "U", upperToothIndices, dfNameIndexMapMerged)
# dfUpperToothRow.to_csv(upperToothRowCentroidsCSV,index=False)
dfUpperToothRow = pd.read_csv(upperToothRowCentroidsCSV)

In [11]:
lowerToothRowCentroidsCSV = r"./data/lowerToothRowCentroids.csv"
# dfLowerToothRow = createToothRowCentroidsDF(RootDir, "L", lowerToothIndices, dfNameIndexMapMerged)
# dfLowerToothRow.to_csv(lowerToothRowCentroidsCSV,index=False)
dfLowerToothRow = pd.read_csv(lowerToothRowCentroidsCSV)

In [12]:
# dfUpperToothRow.isnull()

In [13]:
def getToothRowMeanCentroids(dfToothRow, toothIndices):
    cols2ComputeMean = [str(id)+xyz for id in toothIndices for xyz in ["x","y","z"]] #["11x","11y","11z","12x",...]
    return dfToothRow[cols2ComputeMean].mean(axis=0,skipna=True).to_numpy().reshape(-1,3)

initMeanToothRow = getToothRowMeanCentroids(dfUpperToothRow, upperToothIndices)

In [14]:
def findOptimalRigidTransform(pRef,pMov):
    """已知pRef,pMov点的对应关系，计算最优刚性变换对应的旋转矩阵R和平移向量t"""
    """pRef.shape=(M,3) pMov.shape=(M,3): R,t = argmin 1/M * sum(||Qi @ R + t - Pi||_2)"""
    assert pRef.shape == pMov.shape
    meanP = pRef.mean(axis=0)
    meanQ = pMov.mean(axis=0)
    P = pRef - meanP
    Q = pMov - meanQ
    H = Q.T @ P # shape=(3,3)
    u, s, vh = np.linalg.svd(H)
    D = np.diag([1.,1.,np.linalg.det(u @ vh)])
    R = u @ D @ vh
    t = meanP - meanQ @ R
    tpMov = pMov @ R + t
    return R, t, tpMov

def findOptimalIsoScaledRigidTransform(pRef,pMov):
    """已知pRef,pMov点的对应关系，计算最优缩放的刚性变换对应的缩放常数c,旋转矩阵R和平移向量t"""
    """pRef.shape=(M,3) pMov.shape=(M,3): R,t = argmin 1/M * sum(||Qi @ (c*R) + t - Pi||_2)"""
    assert pRef.shape == pMov.shape
    meanP = pRef.mean(axis=0)
    meanQ = pMov.mean(axis=0)
    P = pRef - meanP
    Q = pMov - meanQ
    n, dim = P.shape
    H = 1./n * Q.T @ P # shape=(3,3)
    u, s, vh = np.linalg.svd(H)
    D = np.diag([1.,1.,np.linalg.det(u @ vh)])
    R = u @ D @ vh
    varQ = np.var(Q, axis=0).sum()
    c = 1/varQ * np.sum(s) # scale factor
    t = meanP - meanQ @ (c*R)
    tpMov = pMov @ (c*R) + t
    return c, R, t, tpMov

def getRigidTransformError(pRef, pMov, R, t):
    error = pMov @ R + t - pRef
    return np.linalg.norm(error,ord="fro")

def getIsoScaledRigidTransformError(pRef, pMov, c, R, t):
    error = pMov @ (c*R) + t - pRef
    return np.linalg.norm(error,ord="fro")

In [16]:
pMov = dfUpperToothRow.iloc[1,7:].to_numpy().reshape(-1,3).astype(np.float64)
pRef = initMeanToothRow
R, t, _ = findOptimalRigidTransform(pRef,pMov)
initError = getRigidTransformError(pRef, pMov, np.eye(3), np.array([0.,0.,0.]))
finalError = getRigidTransformError(pRef, pMov, R, t)
print("rigid trans error: ",finalError)

rigid trans error:  6.482428508539035


In [17]:
s, R, t, _ = findOptimalIsoScaledRigidTransform(pRef,pMov)
print("iso scaled rigid trans error: ", getIsoScaledRigidTransformError(pRef, pMov, s, R, t))

iso scaled rigid trans error:  6.211578011214068


In [18]:
def getToothRows(dfToothRow, toothIndices):
    rowCount = dfToothRow.shape[0]
    posCols = [str(id)+xyz for id in toothIndices for xyz in ["x","y","z"]]
    toothRows = []
    for i in range(rowCount):
        pg = dfToothRow.loc[i,posCols].to_numpy().astype(np.float64).reshape(-1,3)
        toothRows.append(pg)
    return toothRows #toothRow may contain np.nan

def getOptimalMeanToothRow(toothRows, relDiffEps, max_iter=10):
    """计算平均牙列每颗牙齿重心的位置"""
    initMeanToothRow = np.nanmean(np.array(toothRows),axis=0)
    rowCount = len(toothRows)
    pMasks = [~np.isnan(x)[:,0] for x in toothRows]
    alignedToothRows = [x.copy() for x in toothRows] #toothRow may contain np.nan
    diff = np.inf
    pRef = initMeanToothRow
    prevPRef = pRef.copy()
    iter = 0
    while diff > relDiffEps and iter < max_iter:
        iter += 1
        print("iteration:{},diff:{}".format(iter,diff))
        for i in range(rowCount):
            mask = pMasks[i]
            pMov = alignedToothRows[i]
            R, t, tpMov = findOptimalRigidTransform(pRef[mask,:],pMov[mask,:]) # 刚性配准
            alignedToothRows[i][mask,:] = tpMov
        pRef = np.nanmean(np.array(alignedToothRows),axis=0)
        diff = np.linalg.norm(pRef-prevPRef, ord="fro")/np.linalg.norm(pRef, ord="fro")
        prevPRef = pRef.copy()
    return pRef

def rePositionMeantoothRow(meanToothRow, isUpper=True):
    """将平均牙列的重心作为原点，重心到两颗第一切牙重心连线中心的方向为z轴正向，左侧最后一颗牙到右侧最后一颗牙重心的方向作为近似x轴，OZ叉乘近似x轴得出y轴，OY叉乘OZ得出x轴"""
    """meanToothRow.shape=(14,3) OR (16,3)"""
    toothRow = meanToothRow.copy()
    n, dim = meanToothRow.shape
    OX = np.array([1.,0.,0.])
    OY = np.array([0.,1.,0.])
    OZ = np.array([0.,0.,1.])
    meanCentroid = meanToothRow.mean(axis=0)
    toothRow = toothRow - meanCentroid #原点平移至牙列重心
    
    firstLeftCentroid = meanToothRow[0,:] if isUpper else meanToothRow[n//2,:]
    firstRightCentroid = meanToothRow[n//2,:] if isUpper else meanToothRow[0,:]
    rotatedOZ = (firstLeftCentroid + firstRightCentroid)/2.0 - meanCentroid
    rotatedOZ = rotatedOZ / np.linalg.norm(rotatedOZ, ord=2) #单位化
    angle = np.arccos(np.dot(OZ, rotatedOZ))
    rotAxis = np.cross(rotatedOZ, OZ)
    rotAxis = rotAxis / np.linalg.norm(rotAxis, ord=2)
    K = np.array([[0., -rotAxis[2], rotAxis[1]],\
        [rotAxis[2], 0., -rotAxis[0]],\
        [-rotAxis[1], rotAxis[0], 0.]])
    LeftRotMat = np.identity(3) + np.sin(angle) * K + (1.-np.cos(angle)) * (K @ K)
    RotMat = LeftRotMat.T
    toothRow = toothRow @ RotMat #oz轴旋转至[0.,0.,1.]位置
    
    lastRightCentroid = meanToothRow[-1,:] if isUpper else meanToothRow[n//2-1,:]
    lastLeftCentroid = meanToothRow[n//2-1,:] if isUpper else meanToothRow[-1,:]
    approRotatedOX = lastRightCentroid - lastLeftCentroid
    approRotatedOX = approRotatedOX / np.linalg.norm(approRotatedOX, ord=2) #单位化
    rotatedOY = np.cross(rotatedOZ, approRotatedOX)
    rotatedOY = rotatedOY @ RotMat[:3,:3]
    rotatedOY = rotatedOY / np.linalg.norm(rotatedOY,ord=2)
    
    angle_ROZ = np.arccos(np.dot(OY, rotatedOY))
    if np.cross(rotatedOY, OY)[2]<0:
        angle_ROZ = -angle_ROZ
    rotOZMat = np.array([[np.cos(angle_ROZ),np.sin(angle_ROZ),0.],[-np.sin(angle_ROZ),np.cos(angle_ROZ),0.],[0.,0.,1.]])
    toothRow = toothRow @ rotOZMat
    return toothRow 

In [19]:
def alignToothRows(pRef, toothRows):
    scaleFactors = []
    rotMatrices = []
    transVecs = []
    scaledAlignedToothRows = []
    pMasks = [~np.isnan(x)[:,0] for x in toothRows]
    for pMov,mask in zip(toothRows,pMasks):
        s, R, t, tpMov = findOptimalIsoScaledRigidTransform(pRef[mask,:],pMov[mask,:])
        scaleFactors.append(s)
        rotMatrices.append(R)
        transVecs.append(t)
        tpMovGeneral = pMov.copy()
        tpMovGeneral[mask,:] = tpMov
        scaledAlignedToothRows.append(tpMovGeneral)
    return scaledAlignedToothRows, scaleFactors, rotMatrices, transVecs

In [20]:
def getTransVecPerTooth(initToothRow, s, R, t):
    """将牙列比例缩放换算成每颗牙齿重心的位移"""
    scaledAlignedToothRow = initToothRow * s @ R + t
    ts = scaledAlignedToothRow - initToothRow @ R
    return list(ts) # List of translation vectors

In [21]:
# ts = getTransVecPerTooth(upperToothRows[0], scaleFactors[0], rotMatrices[0], transVecs[0])

In [22]:
# err = upperToothRows[0]*scaleFactors[0]@rotMatrices[0]+transVecs[0] - scaledAlignedUpperToothRows[0]

In [23]:
def getToothPointGroups(tag, srcDir, toothIndices):
    """按照toothIndices顺序读取tag对应的每颗牙齿的点云"""
    pgFiles = [os.path.join(srcDir,str(tID),tag+".txt") for tID in toothIndices]
    existFlags = [os.path.exists(f) for f in pgFiles]
    pgs = [np.loadtxt(f) if os.path.exists(f) else np.nan for f in pgFiles]
    return pgs, existFlags

In [24]:
def createToothPGsForSSA(dfToothRow, toothIndices, srcDir, saveDir, rotMatrices, LstTransVecs):
    """创建牙列位置调整后的repaired-txt用于SSA"""
    for tID in toothIndices:
        tDirPath = os.path.join(saveDir, str(tID))
        if os.path.exists(tDirPath):
            shutil.rmtree(tDirPath)
        os.makedirs(tDirPath)
    for tag,R,ts in zip(dfToothRow["tag"], rotMatrices, LstTransVecs):
        pgs, existFlags = getToothPointGroups(tag, srcDir, toothIndices)
        for i,tID in enumerate(toothIndices):
            if existFlags[i] == True:
                assert not np.isnan(ts[i]).any()
                ssaPG = pgs[i] @ R + ts[i]
                saveTxtFile = os.path.join(saveDir, str(tID), tag+".txt")
                np.savetxt(saveTxtFile, ssaPG)
        print("Finish tag: ", tag)

In [25]:
def getRotAngles(RotMats):
    return np.vstack([rotationMatrixToEulerAngles(R) for R in RotMats])

In [26]:
SAVE_DIR = r".\data\ssa-repaired-txt"

In [27]:
upperToothRows = getToothRows(dfUpperToothRow, upperToothIndices)
refUpperToothRow = getOptimalMeanToothRow(upperToothRows, relDiffEps=1e-4, max_iter=5)
refUpperToothRow = rePositionMeantoothRow(refUpperToothRow)

iteration:1,diff:inf
iteration:2,diff:0.004058913229208649


In [28]:
scaledAlignedUpperToothRows, upperScaleFactors, upperRotMatrices, initUpperTransVecs = alignToothRows(refUpperToothRow, upperToothRows) # toothRows may contain nan values
LstUpperTransVecs = [getTransVecPerTooth(toothRow,s,R,t) for toothRow,s,R,t in zip(upperToothRows,upperScaleFactors, upperRotMatrices, initUpperTransVecs)]

In [32]:
def saveToothRowScales2Hdf5(h5File, UorL, scales, transVecShifts, tags):
    """根据tags保存对应的sRtParams"""
    if not os.path.exists(os.path.dirname(h5File)):
        os.makedirs(os.path.dirname(h5File))
    encodedTags = [tag.encode() for tag in tags]
    scales = list(scales)
    with h5py.File(h5File,'w') as f: #每次覆盖写入
        grp = f.create_group("toothRow{}".format(UorL))
        grp.create_dataset("tag", data=encodedTags)
        grp.create_dataset("s", data=np.array(scales, dtype=np.double))
        grp.create_dataset("ts", data=np.array(transVecShifts, dtype=np.double))

In [33]:
upperTransVecShifts = np.array(LstUpperTransVecs)-np.array(initUpperTransVecs)[:,np.newaxis,:]
saveToothRowScales2Hdf5(r"./data/params/scalesOfUpperToothRow.hdf5", "U", upperScaleFactors, upperTransVecShifts, dfUpperToothRow["tag"].to_list())

In [34]:
print("mean scale:",np.array(upperScaleFactors).mean())
print("max scale:",np.array(upperScaleFactors).max())
print("min scale:",np.array(upperScaleFactors).min())
print("Std dev of scale:",np.sqrt(np.var(np.array(upperScaleFactors))))
rotAnglesXYZ = getRotAngles(upperRotMatrices)
print("mean rotation XYZ angles: ",rotAnglesXYZ.mean(axis=0))
print("max rotation XYZ angles: ",rotAnglesXYZ.max(axis=0))
print("min rotation XYZ angles: ",rotAnglesXYZ.min(axis=0))
print("Std dev of rotation XYZ angles: ",np.sqrt(np.var(rotAnglesXYZ, axis=0)))

mean scale: 0.9990334386316926
max scale: 1.177237543865566
min scale: 0.8724790296207059
Std dev of scale: 0.05101920065347013
mean rotation XYZ angles:  [ 0.03975428  0.00144847 -0.00029261]
max rotation XYZ angles:  [0.15847298 0.07871443 0.0839845 ]
min rotation XYZ angles:  [-0.03532782 -0.12692128 -0.05095244]
Std dev of rotation XYZ angles:  [0.02502459 0.02770015 0.02056212]


In [35]:
# createToothPGsForSSA(dfUpperToothRow, upperToothIndices, RootDir, SAVE_DIR, upperRotMatrices, LstUpperTransVecs)

In [36]:
lowerToothRows = getToothRows(dfLowerToothRow, lowerToothIndices)
refLowerToothRow = getOptimalMeanToothRow(lowerToothRows, relDiffEps=1e-4, max_iter=5)
refLowerToothRow = rePositionMeantoothRow(refLowerToothRow, isUpper=False)
scaledAlignedLowerToothRows, lowerScaleFactors, lowerRotMatrices, initLowerTransVecs = alignToothRows(refLowerToothRow, lowerToothRows) # toothRows may contain nan values
LstLowerTransVecs = [getTransVecPerTooth(toothRow,s,R,t) for toothRow,s,R,t in zip(lowerToothRows,lowerScaleFactors, lowerRotMatrices, initLowerTransVecs)]

iteration:1,diff:inf
iteration:2,diff:0.0035529545833956067


In [37]:
lowerTransVecShifts = np.array(LstLowerTransVecs)-np.array(initLowerTransVecs)[:,np.newaxis,:]
saveToothRowScales2Hdf5(r"./data/params/scalesOfLowerToothRow.hdf5", "L", lowerScaleFactors, lowerTransVecShifts, dfLowerToothRow["tag"].to_list())

In [38]:
# createToothPGsForSSA(dfLowerToothRow, lowerToothIndices, RootDir, SAVE_DIR, lowerRotMatrices, LstLowerTransVecs)

## Tooth Row Alignment for tooth data in folder ./dataWithPhoto

In [39]:
DATASET_DIR = r"./dataWithPhoto"
REP_TXT_DIR = os.path.join(DATASET_DIR, "repaired-txt")
SSA_REP_TXT_DIR = os.path.join(DATASET_DIR, "ssa-repaired-txt")
NAME_INDEX_MAPPING_CSV = os.path.join(DATASET_DIR,"nameIndexMapping.csv")
UPPER_CENTROIDS_CSV = os.path.join(DATASET_DIR,"upperToothRowCentroids.csv")
LOWER_CENTROIDS_CSV = os.path.join(DATASET_DIR,"lowerToothRowCentroids.csv")

In [40]:
infoDF = pd.read_csv(NAME_INDEX_MAPPING_CSV)
infoDF.rename(columns={'tag':'uniqueIndex','index':'tag'},inplace=True) 
upperToothRowDF = infoDF.copy()
upperToothRowDF["tag"] = upperToothRowDF["tag"].apply(lambda x:"{}{}".format(x,"U"))
lowerToothRowDF = infoDF.copy()
lowerToothRowDF["tag"] = lowerToothRowDF["tag"].apply(lambda x:"{}{}".format(x,"L"))

In [41]:
# # 计算上牙列中心位置，储存到csv
# upperToothRowDF = createToothRowCentroidsDF(REP_TXT_DIR, "U", upperToothIndices, upperToothRowDF, centroidStartIndex=15)
# upperToothRowDF.to_csv(UPPER_CENTROIDS_CSV,index=False,encoding="utf_8_sig")

In [42]:
# # 计算下牙列中心位置，储存到csv
# lowerToothRowDF = createToothRowCentroidsDF(REP_TXT_DIR, "L", lowerToothIndices, lowerToothRowDF, centroidStartIndex=15)
# lowerToothRowDF.to_csv(LOWER_CENTROIDS_CSV,index=False,encoding="utf_8_sig")

In [43]:
upperToothRowDF = pd.read_csv(UPPER_CENTROIDS_CSV)
lowerToothRowDF = pd.read_csv(LOWER_CENTROIDS_CSV)

In [45]:
upperToothRowCentroids = getToothRows(upperToothRowDF, upperToothIndices)
# refUpperToothRow 前面已经计算过了
_scaledAlignedUpperToothRows, _upperScaleFactors, _upperRotMatrices, _initUpperTransVecs = alignToothRows(refUpperToothRow, upperToothRowCentroids) # toothRows may contain nan values
_LstUpperTransVecs = [getTransVecPerTooth(toothRow,s,R,t) for toothRow,s,R,t in zip(upperToothRowCentroids,_upperScaleFactors, _upperRotMatrices, _initUpperTransVecs)]

In [52]:
print(_scaledAlignedUpperToothRows[0],_upperScaleFactors[0],_upperRotMatrices[0],_initUpperTransVecs[0])

[[ -4.41047784   0.05925293  16.60700087]
 [-11.9949762    0.14564512  12.65740195]
 [-17.63094371  -0.61176257   7.17804135]
 [-20.10944126  -0.45467647  -0.58786907]
 [         nan          nan          nan]
 [-23.22052291   0.11969689 -10.12870044]
 [-28.47623391   1.18551167 -20.00626531]
 [  5.039876     0.50449897  16.64559708]
 [ 11.46328379   0.17569814  11.58091708]
 [ 17.10004855  -0.1006173    6.51758783]
 [ 19.98651216  -0.40582477  -0.70345755]
 [         nan          nan          nan]
 [ 23.16668763  -0.08734769 -10.46838871]
 [ 28.92759501   0.8549918  -19.93999041]] 1.1274584067568585 [[ 9.92242332e-01  9.25938901e-03 -1.23973462e-01]
 [ 2.95941731e-04  9.97043678e-01  7.68362926e-02]
 [ 1.24318414e-01 -7.62769111e-02  9.89306204e-01]] [-0.69333245 -2.4671092  -6.47563472]


In [46]:
_upperTransVecShifts = np.array(_LstUpperTransVecs)-np.array(_initUpperTransVecs)[:,np.newaxis,:]
saveToothRowScales2Hdf5(os.path.join(DATASET_DIR, "params","scalesOfUpperToothRow.hdf5"), "U", _upperScaleFactors, _upperTransVecShifts, upperToothRowDF["tag"].to_list())

In [69]:
createToothPGsForSSA(upperToothRowDF, upperToothIndices, srcDir=REP_TXT_DIR, saveDir=SSA_REP_TXT_DIR, rotMatrices=_upperRotMatrices, LstTransVecs=_LstUpperTransVecs)

Finish tag:  0U
Finish tag:  1U
Finish tag:  2U
Finish tag:  3U
Finish tag:  4U
Finish tag:  5U
Finish tag:  6U
Finish tag:  7U
Finish tag:  8U
Finish tag:  9U
Finish tag:  10U
Finish tag:  11U
Finish tag:  12U
Finish tag:  13U
Finish tag:  14U
Finish tag:  15U
Finish tag:  16U
Finish tag:  17U
Finish tag:  18U
Finish tag:  19U
Finish tag:  20U
Finish tag:  21U
Finish tag:  22U
Finish tag:  23U
Finish tag:  24U
Finish tag:  25U
Finish tag:  26U
Finish tag:  27U
Finish tag:  28U
Finish tag:  29U
Finish tag:  30U
Finish tag:  31U
Finish tag:  32U
Finish tag:  33U
Finish tag:  34U
Finish tag:  35U
Finish tag:  36U
Finish tag:  37U
Finish tag:  38U
Finish tag:  39U
Finish tag:  40U
Finish tag:  41U
Finish tag:  42U
Finish tag:  43U
Finish tag:  44U
Finish tag:  45U
Finish tag:  46U
Finish tag:  47U
Finish tag:  48U
Finish tag:  49U
Finish tag:  50U
Finish tag:  51U
Finish tag:  52U
Finish tag:  53U
Finish tag:  54U
Finish tag:  55U
Finish tag:  56U
Finish tag:  57U
Finish tag:  58U
Finish 

In [49]:
lowerToothRowCentroids = getToothRows(lowerToothRowDF, lowerToothIndices)
# refLowerToothRow 前面已经计算过了
_scaledAlignedLowerToothRows, _lowerScaleFactors, _lowerRotMatrices, _initLowerTransVecs = alignToothRows(refLowerToothRow, lowerToothRowCentroids) # toothRows may contain nan values
_LstLowerTransVecs = [getTransVecPerTooth(toothRow,s,R,t) for toothRow,s,R,t in zip(lowerToothRowCentroids,_lowerScaleFactors, _lowerRotMatrices, _initLowerTransVecs)]

In [50]:
_lowerTransVecShifts = np.array(_LstLowerTransVecs)-np.array(_initLowerTransVecs)[:,np.newaxis,:]
saveToothRowScales2Hdf5(os.path.join(DATASET_DIR, "params","scalesOfLowerToothRow.hdf5"), "L", _lowerScaleFactors, _lowerTransVecShifts, lowerToothRowDF["tag"].to_list())
saveToothRowScales2Hdf5(os.path.join(DATASET_DIR, "cpdGpParams","scalesOfLowerToothRow.hdf5"), "L", _lowerScaleFactors, _lowerTransVecShifts, lowerToothRowDF["tag"].to_list())

In [73]:
createToothPGsForSSA(lowerToothRowDF, lowerToothIndices, srcDir=REP_TXT_DIR, saveDir=SSA_REP_TXT_DIR, rotMatrices=_lowerRotMatrices, LstTransVecs=_LstLowerTransVecs)

Finish tag:  0L
Finish tag:  1L
Finish tag:  2L
Finish tag:  3L
Finish tag:  4L
Finish tag:  5L
Finish tag:  6L
Finish tag:  7L
Finish tag:  8L
Finish tag:  9L
Finish tag:  10L
Finish tag:  11L
Finish tag:  12L
Finish tag:  13L
Finish tag:  14L
Finish tag:  15L
Finish tag:  16L
Finish tag:  17L
Finish tag:  18L
Finish tag:  19L
Finish tag:  20L
Finish tag:  21L
Finish tag:  22L
Finish tag:  23L
Finish tag:  24L
Finish tag:  25L
Finish tag:  26L
Finish tag:  27L
Finish tag:  28L
Finish tag:  29L
Finish tag:  30L
Finish tag:  31L
Finish tag:  32L
Finish tag:  33L
Finish tag:  34L
Finish tag:  35L
Finish tag:  36L
Finish tag:  37L
Finish tag:  38L
Finish tag:  39L
Finish tag:  40L
Finish tag:  41L
Finish tag:  42L
Finish tag:  43L
Finish tag:  44L
Finish tag:  45L
Finish tag:  46L
Finish tag:  47L
Finish tag:  48L
Finish tag:  49L
Finish tag:  50L
Finish tag:  51L
Finish tag:  52L
Finish tag:  53L
Finish tag:  54L
Finish tag:  55L
Finish tag:  56L
Finish tag:  57L
Finish tag:  58L
Finish 